In [ ]:
def clusteriza_ncm(imp_fert, n_clusters=4, cols_base=['log_kg','log_fob']):
    
    from sklearn.preprocessing import MinMaxScaler as MMS
    MMS=MMS()

    a=0
    for c in cols_base:
        imp_fert.insert(0,'s_'+str(c),MMS.fit_transform(imp_fert[[c]]))
        a+=1
    
    
    from sklearn.cluster import KMeans as KM
    KM=KM(n_clusters=n_clusters)
    
    col_k_s=imp_fert.columns[:a]
    
    q=pd.DataFrame()
    for ncm in imp_fert.CO_NCM.unique():
        imp_fert_t=imp_fert[imp_fert.CO_NCM==ncm]
        imp_fert_t=imp_fert_t.assign(km=KM.fit_predict(imp_fert_t[col_k_s]))
        q=pd.concat([q,imp_fert_t])
    
    imp_fert=q.copy()
    
    # Renomeando os clusters de acordo com o rank entre eles
    q=pd.DataFrame()
    for ncm in imp_fert.CO_NCM.unique():
        t=imp_fert[imp_fert.CO_NCM==ncm]
        t1=t.groupby(by=['CO_NCM','km']).KG_LIQUIDO.mean().reset_index().sort_values('KG_LIQUIDO')
        t1=t1.assign(km2=np.arange(0,len(t1)))
        q=pd.concat([q,t1])
    q=q.drop(columns='KG_LIQUIDO')
    imp_fert=imp_fert.merge(q)
    
    imp_fert=imp_fert.drop(columns='km').rename(columns={'km2':'km'})

    imp_fert=imp_fert.sort_values('nick')
    
    plt.show()

    return imp_fert

print("** Função clusteriza_ncm(imp_fert, n_clusters=4, cols_base=['s_log_kg','s_log_fob']) disponível **")